In [1]:
# will always need this 4 lines to start 
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext("local", "Foreach")

22/03/02 02:42:54 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/02 02:42:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/02 02:42:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/02 02:42:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/02 02:42:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/03/02 02:42:56 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/03/02 02:42:56 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
# foreach
# apply a function to each element in the rdd (all partitions)
# write any custom function that deal with db, datastore, cache, etc
orders = [
    # SYMBOL QTY
    ('INFY', 200),
    ('TSLA', 50),
    ('EMR', 20),
    ('INFY', 100),
    ('TSLA', 25)
]

def add(acc, value):
    output = acc + value
    print("acc", acc, "value", value, "output", output)
    return output

orderRdd = sc.parallelize(orders)

resultRdd = orderRdd.foldByKey(0, add)
resultRdd.collect()

[Stage 0:>                                                          (0 + 1) / 1]acc 0 value 200 output 200
acc 0 value 50 output 50
acc 0 value 20 output 20
acc 200 value 100 output 300
acc 50 value 25 output 75


[('INFY', 300), ('TSLA', 75), ('EMR', 20)]

In [4]:
# resultRdd has result, new we apply foreach, it may store data to data store
def updateDB(stock):
    print("Saving ", str(stock), " to db ")

#foreach is ACTION method
resultRdd.foreach(updateDB)

Saving  ('INFY', 300)  to db 
Saving  ('TSLA', 75)  to db 
Saving  ('EMR', 20)  to db 


In [5]:
# foreachPartition
# custom logic to handle data in the partitions
# runs inside executors
# foreach process 1 element at a time, where foreachPartition can process all
# parition data as bulk
# bulk insert/update/delete

# iterator will have each partition data as a whole
# part 0 - 5 records, then iterator will have 5 records
def processData(iterator):
    print(Process data called
    for record in iterator:
        print("Processing ", str(record))
        
    print('-' * 30)
    
# Action method
resultRdd.foreachPartition(processData)

Processing  ('INFY', 300)
Processing  ('TSLA', 75)
Processing  ('EMR', 20)
------------------------------
